# Variational autoencoders

I am interested in analysing encoded vectors of images and text.
My understanding of neural networks is lacking, so my experiments will deal with basic questions on what makes VAEs work and how to scale them properly.
These are my results and realizations.

I started with the [Lamb et al.](https://github.com/vdumoulin/discgen) VAE architecture mentioned in [Decoding The Thought Vector](http://gabgoh.github.io/ThoughtVectors/).

## First questions

In my first attempts the model was able to overfit a tiny dataset of 35 images.
The images were downscaled to 16x8. The training time skyrockets above a 32x32 resolution.

Anything non-trivial failed:
* Giving it 2,500 images produced blurry colored scenes.
* A larger model produced blurry colors even after 150 epochs, after the loss drop slowed down.

The hyperparameters that I could change are:

`
number of filters, encoded dimension, number of epochs, dataset size
`

I want to train a larger model but I need a proof that it would learn at all.

### Epoch size

Initially, I was not sure whether an optimization algorithm's memory persists between epochs. Maybe momentum resets? The learning seems faster when it is not disrupted every minute to display its progress.

<img src="files/dataset_size.png">

This turned out to be subjective. The losses are identical after the same number of iterations.

---

In all tests I standardize the dataset size to 100,000 samples, repeating the data if it is smaller.
Without this, the learning speeds between datasets are incomparable.

# Video dataset

I wanted to train the VAE on video frames, but this has its problems.

These are results from training on Steven Universe S01E01 as a 16x8 video of ~12,000 frames:

| <img src="files/generated_image_2.png"> | <img src="files/garnet.jpg"> |
|:---:|:---:|

In epoch 2 it already generates "characters". This may be because both Garnet and Steven are a "red body, a head, and black hair."

<img src="files/su.jpg">

The overall decoding turned out well.

### The sample ratio problem

The (x=3, y=2) frame from top-left is where it fails even after 500 epochs. The shot lasts 32 frames in the animation. It is mostly purple, which the VAE did not learn to generate at all.

This is very likely to happen in videos. Suppose the dataset was a 2-hour movie. Some subset of scenes could look very similiar to each other and be present for a total of 40 minutes(72,000 frames). A quick, different-looking shot could last a fraction of a second(10 frames). This gives us a >10,000:1 ratio of imagery frequency in the dataset. ADAM's supposed ability to help infrequent weights did not fix this problem.

Datasets of static images don't have this problem.

## VAE Filters
I implemented the [Lamb et al. VAE example](https://github.com/vdumoulin/discgen/blob/master/experiments/train_celeba_vae.py) in keras using the following building block:
```python
def add_block(size):
    for i in range(2):
        encoder.add(Convolution2D(size, 3, padding="same", kernel_initializer=init_w))
        encoder.add(BatchNormalization())
        encoder.add(Activation("relu"))
    encoder.add(Convolution2D(size, 2, strides=2, kernel_initializer=init_w))
    encoder.add(BatchNormalization())
    encoder.add(Activation("relu"))
```
Which I then repeat enough times to get the `(h * 2, h)` input image down to `(4, 2)`:
```python
add_block(filters)
add_block(filters * 2)
# add_block(filters * 4) ...
```
The `filters` parameter should be 32, but I also try other values.

### Computational complexity
This or similiar rate of increase is common in CNN architectures.

The image resolution is halved(area to 1/4) each iteration. The filters number doubles, and since convolutions are calculated using a dense matrix between the filters, this multiplies the number of operations by 4. The total cost does not change. The calculation spends the same amount of time on each layer of abstraction.

# VAE Changes

### New validation metric - reconstruction loss only

I don't care about the quality of the encoded vector, only about the reconstruction. While the VAE loss is decreasing all the time, the reconstruction loss sometimes stops very early. This agrees with what can be seen in the decoded images. The loss may decrease by 2.0 over additional 100 epochs without a visible improvement.

### Ignore sampling randomness during validation

First autoencoded video was very noisy, this way it shows the best reconstruction it can.

## Adam

The Adam optimizer works better than RMSProp.
<img src="files/adam.png">

# Dataset size and number of filters

I tested a combination of:
* filters from `[32, 64, 128]`
* dataset size from `[100, 500, 2500]`
 
<img src="files/d100/loss.png" width="540">
<img src="files/d100/binary_crossentropy.png" width="540">

For a tiny dataset, increasing filters from 32 to 64 gives a much larger improvement than from 64 to 128.

This is especially true for the reconstruction loss, which stops learning after 20 epochs. Its curve fits the growing quality of decoded images better than the VAE loss curve.

| Epoch | Correct image | Decoded image |
|-|-|
| 1 | <img src="files/vae_d100_f64/correct_image.png" width="192"> | <img src="files/vae_d100_f64/decoded_image_1.png" width="192"> |
| 5 | <img src="files/vae_d100_f64/correct_image.png" width="192"> | <img src="files/vae_d100_f64/decoded_image_5.png" width="192"> |
| 10 | <img src="files/vae_d100_f64/correct_image.png" width="192"> | <img src="files/vae_d100_f64/decoded_image_10.png" width="192"> |
| 20 | <img src="files/vae_d100_f64/correct_image.png" width="192"> | <img src="files/vae_d100_f64/decoded_image_20.png" width="192"> |
| 40 | <img src="files/vae_d100_f64/correct_image.png" width="192"> | <img src="files/vae_d100_f64/decoded_image_40.png" width="192"> |
| 60 | <img src="files/vae_d100_f64/correct_image.png" width="192"> | <img src="files/vae_d100_f64/decoded_image_60.png" width="192"> |
| 80 | <img src="files/vae_d100_f64/correct_image.png" width="192"> | <img src="files/vae_d100_f64/decoded_image_80.png" width="192"> |
| 100 | <img src="files/vae_d100_f64/correct_image.png" width="192"> | <img src="files/vae_d100_f64/decoded_image_100.png" width="192"> |

| <img src="files/d100/val_loss.png"> | <img src="files/d100/val_binary_crossentropy.png"> |
|:---:|:---:|

I will not analyze validation measurements because they are either random or overfit.

<img src="files/f128/loss.png" width="540">

(Larger datasets are harder to learn.)

<img src="files/d500/loss.png" width="540">
<img src="files/d2500/loss.png" width="540">

Filter size improves loss more in larger datasets.

I left the `(dataset_size=2500, filters=128)` model running because relatively to its own speed, it was not slowing down before the 100th epoch.  
It also shows my disappointment. A large model training for 2 days learns gray spots.

| Epoch | Correct image | Decoded image |
|-|-|
| 1 | <img src="files/vae_d2500_f128/correct_image.png" width="192"> | <img src="files/vae_d2500_f128/decoded_image_1.png" width="192"> |
| 5 | <img src="files/vae_d2500_f128/correct_image.png" width="192"> | <img src="files/vae_d2500_f128/decoded_image_5.png" width="192"> |
| 10 | <img src="files/vae_d2500_f128/correct_image.png" width="192"> | <img src="files/vae_d2500_f128/decoded_image_10.png" width="192"> |
| 25 | <img src="files/vae_d2500_f128/correct_image.png" width="192"> | <img src="files/vae_d2500_f128/decoded_image_25.png" width="192"> |
| 50 | <img src="files/vae_d2500_f128/correct_image.png" width="192"> | <img src="files/vae_d2500_f128/decoded_image_50.png" width="192"> |
| 100 | <img src="files/vae_d2500_f128/correct_image.png" width="192"> | <img src="files/vae_d2500_f128/decoded_image_100.png" width="192"> |
| 250 | <img src="files/vae_d2500_f128/correct_image.png" width="192"> | <img src="files/vae_d2500_f128/decoded_image_250.png" width="192"> |

<img src="files/d500/binary_crossentropy.png" width="540">
<img src="files/d2500/binary_crossentropy.png" width="540">

Reconstruction loss is already saturated at 64 filters.

## Theory on number of filters


Suppose we didn't train the model, but searched for the best one. [Networks can be compressed](https://www.youtube.com/watch?v=CrDRr2fxbsg) by using only 4 different weight values, so the number of possible 3x3 kernels is:

`
4 ^ (3 * 3) = 4 ^ 9 = 2 ^ 18 = ~256k
`

`(2 2 0)` is just `(1 1 0) * 2`. Many kernels would be linear combinations of other kernels.  
We can just choose `(1 0 0)`, `(0 1 0)` etc. If we only filled the 2x2 square where the kernels don't overlap, we would reach perfect encoding with:

`
2 x 2 (x channels) = 12` filters

This is just a complicated way of saying that if we encode `n` into `>=n` dimensions, we can do so perfectly.

### The numbers

This means a *full* CNN would halve the image using layers with 12, 48, 192 etc. filters.  
My VAE uses 32, 64, 128. That is more than a full CNN in the lower layers!  
It is less near and in the encoded vector because it only doubles. For it to work well with an `f`-times smaller encoded dimension, I have to claim that image data is redundant by a factor of `f`.

*I interpret this so that when a CNN underperforms due to a lack of filters, it happens in the top layers because of loss of information.*  
And not e.g. that the bottom layer can't keep up with producing wildly different textures.

GoogLeNet classifies 224x224x3 images into a (1000,) vector, a factor of ~150.  
Lamb et al. autoencodes 64x64x3 images into a (1000,) vector, a factor of ~12.  
It was demoed on the Celeb-A dataset of faces, so I should expect good results for general imagery to use an even lower ratio(larger model).

# todo
it doesn't work but it should

stochastic noise? batch_size = dataset_size, trivial optimization, sleep

or get like a 10,000 timer bigger dataset